In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical

# specifically for cnn
from tensorflow.keras.layers import Dropout, Flatten,Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [ ]:
#Tensorboard Things

%load_ext tensorboard
import tensorflow as tf
import datetime, os


In [ ]:
#Importing Image Dataset

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fid = drive.ListFile({'q':"title='cervicaldata.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('cervicaldata.zip')

In [ ]:
!unzip cervicaldata.zip

Archive:  cervicaldata.zip
   creating: Dyskeratotic/
  inflating: Dyskeratotic/001_01.bmp  
  inflating: Dyskeratotic/001_02.bmp  
  inflating: Dyskeratotic/001_03.bmp  
  inflating: Dyskeratotic/001_04.bmp  
  inflating: Dyskeratotic/001_05.bmp  
  inflating: Dyskeratotic/002_01.bmp  
  inflating: Dyskeratotic/002_02.bmp  
  inflating: Dyskeratotic/002_03.bmp  
  inflating: Dyskeratotic/002_04.bmp  
  inflating: Dyskeratotic/002_05.bmp  
  inflating: Dyskeratotic/002_06.bmp  
  inflating: Dyskeratotic/002_07.bmp  
  inflating: Dyskeratotic/002_08.bmp  
  inflating: Dyskeratotic/002_09.bmp  
  inflating: Dyskeratotic/002_10.bmp  
  inflating: Dyskeratotic/003_01.bmp  
  inflating: Dyskeratotic/003_02.bmp  
  inflating: Dyskeratotic/003_03.bmp  
  inflating: Dyskeratotic/003_04.bmp  
  inflating: Dyskeratotic/003_05.bmp  
  inflating: Dyskeratotic/003_06.bmp  
  inflating: Dyskeratotic/003_07.bmp  
  inflating: Dyskeratotic/003_08.bmp  
  inflating: Dyskeratotic/003_09.bmp  
  inflatin

In [ ]:
#For the distribution of the dataset
import os
import numpy as np
import glob
import shutil
class DatasetDivision:
	def __init__(self, root_dir="",output_dir=""):
		self.root_dir = root_dir
		self.output_dir = output_dir
		print("Instance of the class created")
	def printnow(self, new_dir):
		print("Just testing that the method calling is working"+new_dir)
	def divide_dataset(self, root_dir,output_dir):
		self.root_dir =root_dir
		self.output_dir = output_dir
		if os.path.exists(self.output_dir):
			if not os.path.exists(os.path.join(self.output_dir,'train')):
				os.mkdir(os.path.join(self.output_dir,'train')) 
				os.mkdir(os.path.join(self.output_dir,'val')) 
				os.mkdir(os.path.join(self.output_dir,'test')) 
		else:
			os.mkdir(self.output_dir)
			os.mkdir(os.path.join(self.output_dir,'train')) 
			os.mkdir(os.path.join(self.output_dir, 'val')) 
			os.mkdir(os.path.join(self.output_dir, 'test'))
		# Split train/val/test sets
		for file in os.listdir(root_dir):            
			classes_path = os.path.join(root_dir, file)  
			class_files = [name for name in glob.glob(os.path.join(classes_path,'*.bmp'))] 
			train_and_valid, test = train_test_split(class_files, test_size=0.20, random_state=42)  #Test data size will be 20%
			train, val = train_test_split(train_and_valid, test_size=0.25, random_state=42)  #Validation data size will be 20% and leaving 60% of data for training #

			#creating the training, validation and testing directories where the data will be moved.
			train_dir = os.path.join(self.output_dir, 'train',file) #creates the train data path for Divided_Dataset 
			val_dir = os.path.join(self.output_dir, 'val', file) #creates the validation data path for Divided_Dataset 
			test_dir = os.path.join(self.output_dir, 'test',file) #creates the test data path for Divided_Dataset 
			if not os.path.exists(train_dir):
				os.mkdir(train_dir)
			if not os.path.exists(val_dir):
				os.mkdir(val_dir)
			if not os.path.exists(test_dir):
				os.mkdir(test_dir)

			for frame_folders in train:
				#get only the last directory of the path frame_folders
				frame_folder = os.path.join(root_dir,file,frame_folders)
				shutil.move(frame_folder,train_dir)
			for frame_folders in val:
				frame_folder = os.path.join(root_dir,file,frame_folders)
				shutil.move(frame_folder,val_dir)
			for frame_folders in test:
				frame_folder = os.path.join(root_dir,file,frame_folders)
				shutil.move(frame_folder,test_dir)
			print('Dataset Division finished.')

In [ ]:
path_dir= "/content/sipakmed/"
output_dir = "/content/sipakmedFormat"
datasetdiv1 = DatasetDivision(path_dir,output_dir)
datasetdiv1.printnow("The new guy")
datasetdiv1.divide_dataset(path_dir, output_dir)

Instance of the class created
Just testing that the method calling is workingThe new guy
Dataset Division finished.
Dataset Division finished.
Dataset Division finished.
Dataset Division finished.
Dataset Division finished.


In [ ]:
pip install git+https://github.com/aleju/imgaug.git

  Cloning https://github.com/aleju/imgaug.git to /tmp/pip-req-build-roodr4z7
  Running command git clone -q https://github.com/aleju/imgaug.git /tmp/pip-req-build-roodr4z7
  Created wheel for imgaug: filename=imgaug-0.4.0-cp36-none-any.whl size=971107 sha256=aaefb1cba47c7f0b012190157b624824dee0934157edf1f54b695d45fde5619e
  Stored in directory: /tmp/pip-ephem-wheel-cache-3_9iv5uv/wheels/65/3d/94/ee32cbeaa29c473a4db74c2d21904ac747311fdca4732665f0
Successfully built imgaug
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.4.0 which is incompatible.
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
#Creating a class for data augmentation
import  cv2
import imgaug
import imageio
import os
import numpy as np
from imgaug import augmenters as iaa


class DataAugmentation:


	def __init__(self, root_dir="",output_dir=""):
		self.root_dir = root_dir
		self.output_dir = output_dir
		print("Instance of the DataAugmentation class created")


	def augmentation_of_image(self, test_image, output_path):
		self.test_image = test_image;
		self.output_path = output_path;
		#define the Augmenters



		#properties: A range of values signifies that one of these numbers is randmoly chosen for every augmentation for every batch

		# Apply affine transformations to each image.
		rotate = iaa.Affine(rotate=(-90,90));  
		scale = iaa.Affine(scale={"x": (0.5, 0.9), "y": (0.5,0.9)}); 
		translation = iaa.Affine(translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)});
		shear = iaa.Affine(shear=(-2, 2)); 
		zoom = iaa.PerspectiveTransform(scale=(0.01, 0.15), keep_size=True) 
		h_flip = iaa.Fliplr(1.0); 
		v_flip = iaa.Flipud(1.0); 
		padding=iaa.KeepSizeByResize(iaa.CropAndPad(percent=(0.05, 0.25)))


		#More augmentations
		blur = iaa.GaussianBlur(sigma=(0, 1.22)) 
		contrast = iaa.contrast.LinearContrast((0.75, 1.5)); 
		contrast_channels = iaa.LinearContrast((0.75, 1.5), per_channel=True) 
		sharpen = iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5));
		gauss_noise = iaa.AdditiveGaussianNoise(scale=0.111*255, per_channel=True) 
		laplace_noise = iaa.AdditiveLaplaceNoise(scale=(0, 0.111*255)) 


		#Brightness 
		brightness = iaa.Multiply((0.35,1.65)) 
		brightness_channels = iaa.Multiply((0.5, 1.5), per_channel=0.75) 

		#CHANNELS (RGB)=(Red,Green,Blue)
		red =iaa.WithChannels(0, iaa.Add((10, 100))) 
		red_rot = iaa.WithChannels(0,iaa.Affine(rotate=(0, 45))) 
		green= iaa.WithChannels(1, iaa.Add((10, 100)))
		green_rot=iaa.WithChannels(1,iaa.Affine(rotate=(0, 45))) 
		blue=iaa.WithChannels(2, iaa.Add((10, 100)))
		blue_rot=iaa.WithChannels(2,iaa.Affine(rotate=(0, 45))) 

		#colors
		channel_shuffle =iaa.ChannelShuffle(1.0); #shuffle all images of the batch
		grayscale = iaa.Grayscale(1.0)
		hue_n_saturation = iaa.MultiplyHueAndSaturation((0.5, 1.5), per_channel=True) #change hue and saturation with this range of values for different values 
		add_hue_saturation = iaa.AddToHueAndSaturation((-50, 50), per_channel=True) #add more hue and saturation to its pixels
		#Quantize colors using k-Means clustering
		kmeans_color = iaa.KMeansColorQuantization(n_colors=(4, 16)) #quantizes to k means 4 to 16 colors (randomly chosen). Quantizes colors up to 16 colors

		#Alpha Blending 
		blend =iaa.AlphaElementwise((0, 1.0), iaa.Grayscale((0,1.0))) ; #blend depending on which value is greater

		#Contrast augmentors
		clahe = iaa.CLAHE(tile_grid_size_px=((3, 21),[0,2,3,4,5,6,7])) #create a clahe contrast augmentor H=(3,21) and W=(0,7)
		histogram = iaa.HistogramEqualization() #performs histogram equalization

		#Augmentation list of metadata augmentors
		OneofRed = iaa.OneOf( [red]);
		OneofGreen = iaa.OneOf( [green] );
		OneofBlue = iaa.OneOf( [blue]);
		contrast_n_shit = iaa.OneOf([contrast, brightness, brightness_channels]);
		SomeAug = iaa.SomeOf(2,[rotate,scale, translation, shear, h_flip,v_flip],random_order=True);
		SomeClahe = iaa.SomeOf(2, [clahe, iaa.CLAHE(clip_limit=(1, 10)),iaa.CLAHE(tile_grid_size_px=(3, 21)),iaa.GammaContrast((0.5, 2.0)), iaa.AllChannelsCLAHE() , iaa.AllChannelsCLAHE(clip_limit=(1, 10), per_channel=True)],random_order=True) #Random selection from clahe augmentors
		edgedetection= iaa.OneOf([iaa.EdgeDetect(alpha=(0, 0.7)),iaa.DirectedEdgeDetect(alpha=(0, 0.7), direction=(0.0, 1.0))]);# Search in some images either for all edges or for directed edges.These edges are then marked in a black and white image and overlayed with the original image using an alpha of 0 to 0.7.
		canny_filter = iaa.OneOf([iaa.Canny(), iaa.Canny(alpha=(0.5, 1.0), sobel_kernel_size=[3, 7])]); #choose one of the 2 canny filter options
		OneofNoise = iaa.OneOf([blur, gauss_noise, laplace_noise])
		Color_1 = iaa.OneOf([channel_shuffle,grayscale, hue_n_saturation , add_hue_saturation, kmeans_color]);
		Color_2 = iaa.OneOf([channel_shuffle,grayscale, hue_n_saturation , add_hue_saturation, kmeans_color]);
		Flip = iaa.OneOf([histogram , v_flip, h_flip]);

		#Define the augmentors used in the DA
		Augmentors= [SomeAug, SomeClahe, edgedetection, canny_filter, Color_1, contrast_n_shit]


		for i in range(0,6):
			img = cv2.imread(test_image) #read you image
			images = np.array([img for _ in range(6)], dtype=np.uint8)  # 6 is the size of the array that will hold 6 different images
			images_aug = Augmentors[i].augment_images(images)  
			cv2.imwrite(os.path.join(output_path,test_image +"new"+str(i)+'.jpg'), images_aug[i]) 

In [ ]:
import shutil

class DataAugmentation_Extension:
	def __init__(self, directory=""):
		self.directory = directory
		print("Instance of DataAugmentation_Extension class created")

	def printnow(self, dir):
		print("Just testing that the method calling is working "+ dir)


	def extend_dataset(self,directory):
		#Create an instance of class 
		print("HEY")
		library_augment= DataAugmentation();
		self.directory = directory 
		if not os.path.exists(self.directory):
			print("ERROR! Couldn't find directory!")
		else:
			print("Directory exists")
		for file in os.listdir(directory):            #for any file inside the root directory 
			classes_path = os.path.join(directory, file)  #So for every folder class we create a class directory
			class_files = [name for name in glob.glob(os.path.join(classes_path,'*.bmp'))]  #alternatively we can use the globe as mentioned
			print(class_files); #call augmentation for all class_files
			for i in range(len(class_files)):
				library_augment.augmentation_of_image(class_files[i], classes_path)

In [ ]:
#data augmentation
#directories
target_directory = "/content/sipakmedFormat/train"
#create an instance of the class
datasetda = DataAugmentation_Extension()
datasetda.extend_dataset(target_directory)

Instance of DataAugmentation_Extension class created
HEY
Instance of the DataAugmentation class created
Directory exists
['/content/sipakmedFormat/train/Intermediate/123_03.bmp', '/content/sipakmedFormat/train/Intermediate/068_05.bmp', '/content/sipakmedFormat/train/Intermediate/053_06.bmp', '/content/sipakmedFormat/train/Intermediate/029_08.bmp', '/content/sipakmedFormat/train/Intermediate/004_10.bmp', '/content/sipakmedFormat/train/Intermediate/015_04.bmp', '/content/sipakmedFormat/train/Intermediate/071_06.bmp', '/content/sipakmedFormat/train/Intermediate/106_04.bmp', '/content/sipakmedFormat/train/Intermediate/057_01.bmp', '/content/sipakmedFormat/train/Intermediate/050_02.bmp', '/content/sipakmedFormat/train/Intermediate/035_06.bmp', '/content/sipakmedFormat/train/Intermediate/102_08.bmp', '/content/sipakmedFormat/train/Intermediate/035_01.bmp', '/content/sipakmedFormat/train/Intermediate/090_01.bmp', '/content/sipakmedFormat/train/Intermediate/075_02.bmp', '/content/sipakmedForma

**Feature Combination**

In [ ]:
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


"""LOAD DATA"""
# path to folder with data
path = "/content/sipakmedFormat/"
# expected data structure inside folder: train, test, val. in each folder: one folder for each class,
# comprising its respective slide images.


# specify image data generator with data augmentation (train_datagen) resp. without (no_DA_IDG)
train_datagen = ImageDataGenerator(featurewise_center=False,
                                   rotation_range = 5, fill_mode="nearest",
                                   zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0, # occasionally out of range
                                   horizontal_flip = True, vertical_flip=True,
                                   brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()

# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)

training_set_V2 =  no_DA_IDG.flow_from_directory(path+"train/", ### TO USE FOR FEATURE EXTRACTION
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical', shuffle=False)

validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)

validation_set_V2 = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""Loading Models & their respective weights"""
from keras.models import Sequential, load_model
model_vgg = load_model('model_vgg16.h5') 
model_xcep = load_model('model_xcep.h5')
model_vgg19 = load_model('model_vgg19.h5') 
model_res50 = load_model('model_Res50.h5')

model_xcep.load_weights("K_XCEP_T2.h5")
model_vgg.load_weights("K_VGG_T2.h5")
model_res50.load_weights("K_R50_T2.h5")
model_vgg19.load_weights("K_VGG19_T2.h5")



"""MODEL COMBINATION"""

vgg_extractor = Model(inputs=model_vgg.input, outputs=model_vgg.get_layer("dense_1024").output)
xcep_extractor = Model(inputs=model_xcep.input, outputs=model_xcep.get_layer("dense_1024").output)
vgg19_extractor = Model(inputs=model_vgg19.input, outputs=model_vgg19.get_layer("dense_1024").output)
r50_extractor = Model(inputs=model_res50.input, outputs=model_res50.get_layer("dense_1024").output)

y_train = to_categorical(training_set_V2.classes)
X_train_m1 = vgg_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m2 = xcep_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m3 = vgg19_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m4 = r50_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train = np.concatenate([X_train_m1, X_train_m2, X_train_m3, X_train_m4], axis=1)

y_val = to_categorical(validation_set_V2.classes)
X_val_m1 = vgg_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m2 = xcep_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m3 = vgg19_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m4 = r50_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val = np.concatenate([X_val_m1, X_val_m2, X_val_m3, X_val_m4], axis=1)

y_test = to_categorical(test_set_V2.classes)
X_test_m1 = vgg_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m2 = xcep_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m3 = vgg19_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m4 = r50_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test = np.concatenate([X_test_m1, X_test_m2, X_test_m3, X_test_m4], axis=1)


"""TRAIN & TEST FEATURE EXTRACTION MODEL"""

np.random.seed(668)

opt = Adam(learning_rate=1e-3)
model = Sequential()
model.add(Dropout(0.5, input_shape=(4096,)))
model.add(BatchNormalization())
model.add(Dense(5, activation="softmax"))
model1=model
model1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])


epochs = 200
history1=model1.fit(X_train, y_train, batch_size=16, epochs=epochs, verbose=2, shuffle=True, validation_data=(X_val, y_val))

"""EVALUATE MODEL ON TEST DATA"""
y_test = np.argmax(y_test, axis=1)
pred = np.argmax(model1.predict(X_test), axis=1)
model1.save('model_vgg16_vgg19_Res50_xception.h5')
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

Found 16982 images belonging to 5 classes.
Found 16982 images belonging to 5 classes.
Found 811 images belonging to 5 classes.
Found 811 images belonging to 5 classes.
Found 812 images belonging to 5 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
Epoch 1/200
1062/1062 - 3s - loss: 0.2501 - acc: 0.9244 - val_loss: 0.0248 - val_acc: 0.9938
Epoch 2/200
1062/1062 - 3s - loss: 0.2297 - acc: 0.9345 - val_loss: 0.0193 - val_acc: 0.9914
Epoch 3/200
1062/1062 - 3s - loss: 0.2253 - acc: 0.9383 - val_loss: 0.0194 - val_acc: 0.9926
Epoch 4/200
1062/1062 - 3s - loss: 0.2174 - acc: 0.9417 - val_loss: 0.0287 - val_acc: 0.9901
Epoch 5/200
1062/1062 - 3s - loss: 0.2220 - acc: 0.9408 - val_loss: 0.0359 - val_acc: 0.9901
Epoch 6/200
1062/1062 - 2s - loss: 0.2156 - acc: 0.9424 - val_loss: 0.0256 - val_acc: 0.9938
Epoch 7/200
1062/1062 - 2s - loss: 0.2207 - acc: 0.9435 - val_loss: 0.0275 - val_acc: 0.9914
Epoch 8/200
1062/1062 - 3s - loss: 0.2287 - acc: 0.9408 - va